In [ ]:
import praw
from datetime import datetime
import pandas as pd
import requests

# Define the subreddit and the number of posts you want to retrieve
subreddit_name = "all"
search_item="Healthcare"
num_posts = 100000
num_posts_tosave=10000
Saving_path="your own project folder to save data"

# Define your Reddit API credentials, please replace your own
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
user_agent = "YOUR_USER_AGENT"
username = "YOUR_REDDIT_USERNAME"
password = "YOUR_REDDIT_PASSWORD"

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent,
    username=username,
    password=password,
)

# Define the start and end timestamps (timestamps are in UTC)
start_time = datetime(2015, 1, 1, 0, 0, 0)
end_time = datetime(2023, 10, 15, 0, 0, 0)

# Create a list to store post data
post_data = []

# Use PRAW to retrieve posts from the subreddit and collect data
# Collect posts from top_posts
top_posts = reddit.subreddit(subreddit_name).top(limit=num_posts)

# Collect posts from search_results
search_results_relevance = reddit.subreddit('all').search(search_item, limit=num_posts, sort="relevance")
search_results_hot = reddit.subreddit('all').search(search_item, limit=num_posts, sort="hot")
search_results_top = reddit.subreddit('all').search(search_item, limit=num_posts, sort="top")
search_results_new = reddit.subreddit('all').search(search_item, limit=num_posts, sort="new")

# Combine the results into a single list
combined_results = list(top_posts) + list(search_results_relevance)+ list(search_results_hot)+ list(search_results_top)+ list(search_results_new)

for post in combined_results:
    post_date = datetime.utcfromtimestamp(post.created_utc)
    if start_time <= post_date <= end_time:
        post_date_formatted = post_date.strftime('%Y-%m-%d %H:%M:%S')
        post_id = post.id  # Get the unique post ID
        post_title = post.title
        post_content = post.selftext  # Get the post's self-text
        post_author = post.author.name if post.author else "Unknown"
        image_url = post.url
        # Create a list to store comments for each post
        comments_data = []
        for comment in post.comments.list():
            try:
                # Check if the comment is a regular comment (not "MoreComments")
                if isinstance(comment, praw.models.Comment):
                    com = {}
                    com_author = comment.author.name if comment.author else "Unknown"
                    com['author'] = str(com_author)
                    com['body'] = comment.body
                    com['created_utc'] = datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                    com['distinguished'] = comment.distinguished
                    com['edited'] = comment.edited
                    com['id'] = comment.id
                    com['is_submitter'] = comment.is_submitter
                    com['link_id'] = comment.link_id
                    com['parent_id'] = comment.parent_id
                    com['permalink'] = comment.permalink
                    com['score'] = comment.score
                    com['stickied'] = comment.stickied
                    com['subreddit_id'] = comment.subreddit_id
                    comments_data.append(com)
            except AttributeError:
                # Handle MoreComments objects
                pass
        # Append each collection         
        post_data.append({
            "Text_ID": post_id,
            "Title": post_title,
            "Content": post_content,
            "Date": post_date_formatted,
            "User": post_author,
            "Image URL": image_url,
            "Comments": comments_data
        })
        if len(post_data) >= num_posts_tosave:
            pd.DataFrame(post_data).to_csv(Saving_path + subreddit_name + "_combined.csv", index=False)
            break  # This will exit the loop after collecting the desired number of posts

# Create a Pandas DataFrame from the collected data
pd.DataFrame(post_data).to_csv(Saving_path + subreddit_name +"_combined.csv", index=False)
# Display the DataFrame
display(pd.DataFrame(post_data).shape, pd.DataFrame(post_data).head())
